<a href="https://colab.research.google.com/github/vifirsanova/compling/blob/main/nmt_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`Курс "Компьютерная лингвистика" | НИУ ВШЭ Санкт-Петербург 2024 (c) В.И. Фирсанова`

# Нейросетевой машинный перевод: подготовка данных к машинному обучению

План занятия:

I. Знакомимся с процессом подготовки данных к машинному обучению

II. Вместе проходим туториал TensorFlow и осваиваем модель кодер-декодер на примере рекуррентной нейросети

Загрузка данных

In [1]:
!wget https://raw.githubusercontent.com/vifirsanova/compling/main/data/toy_data.en
!wget https://raw.githubusercontent.com/vifirsanova/compling/main/data/toy_data.ru

--2024-02-08 13:31:58--  https://raw.githubusercontent.com/vifirsanova/compling/main/data/toy_data.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1590 (1.6K) [text/plain]
Saving to: ‘toy_data.en.4’

toy_data.en.4       100%[===================>]   1.55K  --.-KB/s    in 0s      

2024-02-08 13:31:58 (30.8 MB/s) - ‘toy_data.en.4’ saved [1590/1590]

--2024-02-08 13:31:58--  https://raw.githubusercontent.com/vifirsanova/compling/main/data/toy_data.ru
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2501 (2.4K) [text/plain]
Saving t

Импорт библиотек

In [2]:
import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf

Запись данных в переменные

In [3]:
def load_data(path):
  with open(path, 'r', encoding='utf-8') as f:
    return np.array(f.read().split('\n'))

In [4]:
x_data, y_data = load_data('toy_data.ru'), load_data('toy_data.en')

print("Данные языка X:\n", x_data[5])
print("Данные языка Y:\n", y_data[5])

Данные языка X:
     Я люблю тебя.
Данные языка Y:
     I love you.


Создаем выборки

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data)

print("Тестовые данные языка X:\n", x_test)
print("Тестовые данные языка Y:\n", y_test)

Тестовые данные языка X:
 ['    Давай встретимся завтра.'
 '    Что ты обычно говоришь в таких ситуациях?'
 '    Что ты любишь делать в свободное время?'
 '    Что ты хочешь сказать этим?' '    Какие книги ты читаешь?'
 '    Каков твой родной язык?'
 '    Что ты хотел бы заказать в этом ресторане?'
 '    Что ты хочешь сказать?' '    Какие у тебя планы на будущее?'
 '    Я не понимаю.' '    Что ты думаешь о этом месте?'
 '    Какой прекрасный день!' '    Какие фильмы ты любишь смотреть?']
Тестовые данные языка Y:
 ["    Let's meet tomorrow."
 '    What do you usually say in such situations?'
 '    What do you like to do in your free time?'
 '    What do you mean by that?' '    What books do you read?'
 '    What is your native language?'
 '    What would you like to order in this restaurant?'
 '    What do you want to say?' '    What are your plans for the future?'
 "    I don't understand." '    What do you think about this place?'
 '    What a beautiful day!' '    What movies do you l

Создаем объекты Dataset

**Тензор** - объект векторного пространства V конечной размерности n

*shape* - размерность тензора

*dtype* - какие данные хранятся в объекте

**Батч** - некий отрезок данных; на батчи обычно деляться данные для машинного обучения

In [25]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(64).shuffle(len(x_data))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64).shuffle(len(x_data))

for x_strings, y_strings in train_dataset.take(1):
  print("Образец tf-датасета, данные языка X:\n", x_strings[1:3])
  print("\nОбразец tf-датасета, данные языка Y:\n", y_strings[1:3])

Образец tf-датасета, данные языка X:
 tf.Tensor(
[b'    \xd0\xa7\xd1\x82\xd0\xbe \xd1\x82\xd1\x8b \xd0\xb4\xd0\xb5\xd0\xbb\xd0\xb0\xd0\xb5\xd1\x88\xd1\x8c \xd0\xb2\xd0\xb5\xd1\x87\xd0\xb5\xd1\x80\xd0\xbe\xd0\xbc?'
 b'    \xd0\x9a\xd0\xb0\xd0\xba\xd0\xb8\xd0\xb5 \xd1\x83 \xd1\x82\xd0\xb5\xd0\xb1\xd1\x8f \xd1\x85\xd0\xbe\xd0\xb1\xd0\xb1\xd0\xb8?'], shape=(2,), dtype=string)

Образец tf-датасета, данные языка Y:
 tf.Tensor([b'    What do you do in the evening?' b'    What are your hobbies?'], shape=(2,), dtype=string)


Предобработка данных: чистка и токенизация

In [29]:
def tokenize(text):
  # к нижнему регистру
  text_lower = tf.strings.lower(text, encoding='utf-8')
  # оставим знаки препинания и все буквы латиницы и кириллицы
  text_clean = tf.strings.regex_replace(text_lower, '[^ a-zа-я.?!,]', '')
  # добавим пробел перед знаками препинания (для токенизации знаков)
  text_punct = tf.strings.regex_replace(text_clean, '[.?!,]', r' \0 ')
  # избавимся от лишних пробелов
  text = tf.strings.strip(text_punct)
  # добавим метки начала (sos: start of the sentence) и конца (eos: end of the sentence) предложений
  return tf.strings.join(['<SOS>', text, '<EOS>'], separator=' ')

In [32]:
print("Образец токенизации:\n", tokenize(tf.constant("Как твои дела?")).numpy().decode())

Образец токенизации:
 <SOS> как твои дела ? <EOS>


Кодирование (векторизация): готовим данные для "скармливания" их модели

In [67]:
x_text_processor = tf.keras.layers.TextVectorization(standardize=tokenize, ragged=True)

x_text_processor.adapt(train_dataset.map(lambda x, target: x))
print("Образец обучающего словаря языка X:\n", x_text_processor.get_vocabulary()[:10])
print("\nОбразец закодированного текста X:\n\tТекст:", x_strings[1].numpy().decode(), "\n\tКодирование:", x_text_processor(x_strings[1]))

y_text_processor = tf.keras.layers.TextVectorization(standardize=tokenize, ragged=True)

y_text_processor.adapt(train_dataset.map(lambda x, target: target))
print("\nОбразец обучающего словаря языка Y:\n", y_text_processor.get_vocabulary()[:10])
print("\nОбразец закодированного текста X:\n\tТекст:", y_strings[1].numpy().decode(), "\n\tКодирование:", y_text_processor(y_strings[1]))

Образец обучающего словаря языка X:
 ['', '[UNK]', '<SOS>', '<EOS>', '?', 'ты', 'как', 'что', 'тебя', 'у']

Образец закодированного текста X:
	Текст:     Что ты делаешь вечером? 
	Кодирование: tf.Tensor([ 2  7  5 28 89  4  3], shape=(7,), dtype=int64)

Образец обучающего словаря языка Y:
 ['', '[UNK]', '<SOS>', '<EOS>', '?', 'you', 'what', 'do', 'your', 'how']

Образец закодированного текста X:
	Текст:     What do you do in the evening? 
	Кодирование: tf.Tensor([ 2  6  7  5  7 23 10 73  4  3], shape=(10,), dtype=int64)


In [70]:
def encode(x, y):
  # обработка текстов (текст -> словарь -> замена слов их ID'шками из словаря)
  x_encoded = x_text_processor(x).to_tensor()
  y_encoded = y_text_processor(y)
  # (x_encoded, y_encoded) -> ((x_encoded, y_input), y_label) для keras.Model.fit
  # Keras принимает (inputs, labels)
  # inputs = (x_encoded, y_input)
  # labels = y_label
  # y_label += следующий за y_input токен
  y_input = y_encoded[:,:-1].to_tensor()
  y_label = y_encoded[:,1:].to_tensor()
  return (x_encoded, y_input), y_label

In [80]:
train = train_dataset.map(encode, tf.data.AUTOTUNE)
test = test_dataset.map(encode, tf.data.AUTOTUNE)

for (x_encoded_token, y_input_token), y_label_token in train.take(1):
  print("Закодированный текст x:")
  print(x_encoded_token[1, :10].numpy())
  print("\nЗакодированный текст y:")
  print(y_input_token[1, :10].numpy())
  print("\nТот же текст y мы сдвинули на 1 токен вперед:")
  print(y_label_token[1, :10].numpy())

Закодированный текст x:
[ 2  7  5 28 89  4  3  0  0  0]

Закодированный текст y:
[ 2  6  7  5  7 23 10 73  4  0]

Тот же текст y мы сдвинули на 1 токен вперед:
[ 6  7  5  7 23 10 73  4  3  0]


Задание:

1. **По желанию**

* Ознакомиться с [туториалом TensorFlow](https://www.tensorflow.org/text/tutorials/nmt_with_attention) по машинному переводу

* Адаптировать препроцессинг на основе этого воркбука и обучить модель из туториала TensorFlow на данных `toy_data.ru` и `toy_data.en`

2. **Обязательно**

* Ниже: образец подсчета BLEU-Score для системы машинного перевода. Вспомните, как считается BLEU и опишите своими словами форумулу расчета и интерпретацию результатов. Ответьте на следующие вопросы:

  * Что значит кандидат и референс?
  * Для чего подойдет BLEU-Score: подсчета отдельных переводов или результатов работы системы на целом корпусе?
  * Какие "скоры" считаются высокими, а какие - низкими?
  
* Проанализируйте результаты в ячейках ниже. Порассуждайте:

  * На ваш взгляд, какие слова, контексты вызвали трудности у автоматической системы?
  * Руководствуясь знаниями о том, как работают нейросети, предположите, что вызвало ошибки.

In [ ]:
!pip install evaluate

In [83]:
import evaluate

predictions = [
    "In addition to the usual hockey, there is an underwater and even ice version of this game.", #  Помимо обычного хоккея, существует подводная и даже подлёдная разновидности этой игры.
    "You reap what you sow.", # Что посеешь - то и пожнешь.
    "Also, justice is how we distribute the small resources." # Также, справедливость - это и то, как мы распределяем малые ресурсы.
    ]

references = [
    ["In addition to conventional hockey, there is an underwater and even under-ice variety of this game."], #  Помимо обычного хоккея, существует подводная и даже подлёдная разновидности этой игры.
    ["What goes around comes around."], # Что посеешь - то и пожнешь.
    ["Fairness also has to do with how we distribute scarce resources."], # Также, справедливость - это и то, как мы распределяем малые ресурсы.
    ]

bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)

print(results)

{'bleu': 0.374360558192794, 'precisions': [0.5833333333333334, 0.45454545454545453, 0.3333333333333333, 0.2222222222222222], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 36, 'reference_length': 36}
